# Partie 1.1 - Exploration initiale avec PySpark

**ECF - Titre Professionnel Data Engineer (RNCP35288)**

---

## Contexte

Ce notebook constitue la premiere etape de notre pipeline de traitement des donnees de consommation energetique
de batiments. L'objectif est d'explorer le jeu de donnees brut (`consommations_raw.csv`, environ 7,7 millions de lignes)
afin de comprendre sa structure, identifier les problemes de qualite et preparer les etapes de nettoyage ulterieures.

Le fichier de reference `batiments.csv` contient les informations descriptives de 146 batiments
(type, commune, surface, classe energetique, etc.).

### Objectifs de ce notebook

1. Charger les donnees brutes avec PySpark et analyser le schema infere
2. Calculer des statistiques descriptives par type d'energie
3. Identifier et quantifier systematiquement les defauts de qualite
4. Analyser la repartition des mesures par batiment
5. Produire un rapport d'audit complet de la qualite des donnees

### Defauts attendus dans les donnees

- Formats de dates multiples (ISO, FR dd/mm/yyyy, US mm/dd/yyyy, ISO avec T)
- Valeurs negatives (~0,5%)
- Pics aberrants / outliers >15000 (~1%)
- Doublons (~2%)
- Separateurs decimaux mixtes (virgule au lieu de point, ~12%)
- Valeurs textuelles dans le champ consommation ("erreur", "N/A", "---", "null" - ~1,5%)
- Periodes de donnees manquantes (~1%)

In [2]:
# =============================================================================
# Imports et creation de la session Spark
# =============================================================================

from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import DoubleType, StringType
from pyspark.sql.window import Window
import os

# Creation de la session Spark configuree pour le traitement local
spark = SparkSession.builder \
    .appName("ECF Energie - Exploration initiale") \
    .master("local[*]") \
    .config("spark.driver.memory", "4g") \
    .config("spark.sql.shuffle.partitions", "8") \
    .getOrCreate()

# Reduction du niveau de log pour plus de lisibilite
spark.sparkContext.setLogLevel("WARN")

print(f"Version de Spark : {spark.version}")
print(f"Session Spark initialisee avec succes.")

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
26/02/11 18:22:24 WARN Utils: Your hostname, MacBook-Pro-de-Ihab.local, resolves to a loopback address: 127.0.0.1; using 172.20.10.5 instead (on interface en0)
26/02/11 18:22:24 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
26/02/11 18:22:27 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Version de Spark : 4.1.1
Session Spark initialisee avec succes.


## 1. Chargement des donnees de consommation

Nous chargeons le fichier `consommations_raw.csv` avec l'inference automatique du schema par PySpark.
Le fichier contient environ 7,7 millions de lignes avec les colonnes :
- `batiment_id` : identifiant unique du batiment
- `timestamp` : horodatage de la mesure
- `type_energie` : type d'energie (electricite, gaz, etc.)
- `consommation` : valeur de consommation mesuree
- `unite` : unite de mesure

In [3]:
# =============================================================================
# Chargement du fichier de consommations brutes
# =============================================================================

# Chemin vers les donnees (relatif au dossier notebooks)
DATA_DIR = os.path.join(os.path.dirname(os.path.abspath("__file__")), "..", "data")
CONSOMMATIONS_PATH = os.path.join(DATA_DIR, "consommations_raw.csv")
BATIMENTS_PATH = os.path.join(DATA_DIR, "batiments.csv")

print(f"Chemin des consommations : {os.path.abspath(CONSOMMATIONS_PATH)}")
print(f"Chemin des batiments     : {os.path.abspath(BATIMENTS_PATH)}")

# Chargement avec inference de schema automatique
# header=True pour utiliser la premiere ligne comme noms de colonnes
df_conso = spark.read.csv(
    CONSOMMATIONS_PATH,
    header=True,
    inferSchema=True,
    sep=","
)

# Affichage du schema infere
print("\n" + "="*60)
print("SCHEMA INFERE PAR PYSPARK")
print("="*60)
df_conso.printSchema()

# Affichage des 10 premieres lignes
print("\n" + "="*60)
print("APERCU DES 10 PREMIERES LIGNES")
print("="*60)
df_conso.show(10, truncate=False)

Chemin des consommations : /Users/ihababadi/Downloads/ECF_ENERGIE_BATIMENTS (1)/ecf_energie/data/consommations_raw.csv
Chemin des batiments     : /Users/ihababadi/Downloads/ECF_ENERGIE_BATIMENTS (1)/ecf_energie/data/batiments.csv



SCHEMA INFERE PAR PYSPARK
root
 |-- batiment_id: string (nullable = true)
 |-- timestamp: string (nullable = true)
 |-- type_energie: string (nullable = true)
 |-- consommation: string (nullable = true)
 |-- unite: string (nullable = true)


APERCU DES 10 PREMIERES LIGNES
+-----------+-------------------+------------+------------+-----+
|batiment_id|timestamp          |type_energie|consommation|unite|
+-----------+-------------------+------------+------------+-----+
|BAT0141    |2023-12-21 13:00:00|gaz         |342.34      |kWh  |
|BAT0080    |08/08/2023 13:00   |gaz         |1256.73     |kWh  |
|BAT0122    |06/13/2024 11:00:00|eau         |133.57      |m3   |
|BAT0033    |2023-06-25 00:00:00|eau         |0.23        |m3   |
|BAT0064    |11/29/2024 04:00:00|gaz         |12.26       |kWh  |
|BAT0052    |09/15/2024 18:00:00|gaz         |701.01      |kWh  |
|BAT0097    |2024-01-25 12:00:00|gaz         |444.87      |kWh  |
|BAT0073    |27/03/2023 13:00   |electricite |123.97      |kWh  |


## 2. Analyse du schema infere et problemes de typage

L'inference automatique de PySpark peut poser des problemes lorsque les donnees contiennent
des valeurs heterogenes. En particulier, la colonne `consommation` risque d'etre inferee comme
`string` au lieu de `double` en raison de :
- La presence de valeurs textuelles ("erreur", "N/A", "---", "null")
- L'utilisation de la virgule comme separateur decimal dans certaines lignes

In [4]:
# =============================================================================
# Analyse detaillee du schema et des types de donnees
# =============================================================================

# Nombre total de lignes
nb_total = df_conso.count()
print(f"Nombre total de lignes : {nb_total:,}")
print(f"Nombre de colonnes     : {len(df_conso.columns)}")
print(f"Colonnes               : {df_conso.columns}")

# Verification du type de la colonne consommation
print("\n" + "="*60)
print("TYPES DES COLONNES")
print("="*60)
for col_name, col_type in df_conso.dtypes:
    indicateur = " <-- ATTENTION : devrait etre numeric" if col_name == "consommation" and col_type == "string" else ""
    print(f"  {col_name:25s} -> {col_type}{indicateur}")

# Valeurs distinctes de type_energie
print("\n" + "="*60)
print("VALEURS DISTINCTES DE 'type_energie'")
print("="*60)
df_conso.select("type_energie").distinct().orderBy("type_energie").show(truncate=False)

# Valeurs distinctes de unite
print("\n" + "="*60)
print("VALEURS DISTINCTES DE 'unite'")
print("="*60)
df_conso.select("unite").distinct().orderBy("unite").show(truncate=False)

# Echantillon de valeurs non-numeriques dans la colonne consommation
print("\n" + "="*60)
print("EXEMPLES DE VALEURS PROBLEMATIQUES DANS 'consommation'")
print("="*60)
# Valeurs contenant des virgules (separateur decimal mixte)
df_conso.filter(F.col("consommation").contains(",")).select("consommation").distinct().show(10, truncate=False)

# Valeurs purement textuelles
df_conso.filter(
    ~F.col("consommation").rlike(r"^-?[0-9]+([\.\,][0-9]+)?$")
).select("consommation").distinct().show(20, truncate=False)

Nombre total de lignes : 7,758,868
Nombre de colonnes     : 5
Colonnes               : ['batiment_id', 'timestamp', 'type_energie', 'consommation', 'unite']

TYPES DES COLONNES
  batiment_id               -> string
  timestamp                 -> string
  type_energie              -> string
  consommation              -> string <-- ATTENTION : devrait etre numeric
  unite                     -> string

VALEURS DISTINCTES DE 'type_energie'


+------------+
|type_energie|
+------------+
|eau         |
|electricite |
|gaz         |
+------------+


VALEURS DISTINCTES DE 'unite'


+-----+
|unite|
+-----+
|kWh  |
|m3   |
+-----+


EXEMPLES DE VALEURS PROBLEMATIQUES DANS 'consommation'


+------------+
|consommation|
+------------+
|20,38       |
|19,14       |
|1,28        |
|1148,56     |
|12,83       |
|12,33       |
|206,03      |
|47,60       |
|23,54       |
|9,37        |
+------------+
only showing top 10 rows


+------------+
|consommation|
+------------+
|null        |
|N/A         |
|erreur      |
|---         |
+------------+



## 3. Statistiques descriptives par type d'energie

Pour calculer les statistiques numeriques, nous devons d'abord filtrer les valeurs textuelles
et convertir les virgules en points dans le champ `consommation`, puis caster en `double`.

In [5]:
# =============================================================================
# Statistiques descriptives par type d'energie
# =============================================================================

# Preparation : filtrer les valeurs textuelles et convertir les virgules en points
df_conso_num = df_conso.filter(
    # On ne garde que les lignes dont la consommation ressemble a un nombre
    F.col("consommation").rlike(r"^-?[0-9]+([\.\,][0-9]+)?$")
).withColumn(
    # Remplacement de la virgule par un point pour les separateurs decimaux
    "consommation_clean",
    F.regexp_replace(F.col("consommation"), ",", ".").cast(DoubleType())
)

print(f"Lignes avec valeur numerique valide : {df_conso_num.count():,} / {nb_total:,}")

# Statistiques descriptives par type d'energie
print("\n" + "="*60)
print("STATISTIQUES DESCRIPTIVES PAR TYPE D'ENERGIE")
print("="*60)

stats_par_energie = df_conso_num.groupBy("type_energie").agg(
    F.count("consommation_clean").alias("nb_mesures"),
    F.round(F.mean("consommation_clean"), 2).alias("moyenne"),
    F.round(F.stddev("consommation_clean"), 2).alias("ecart_type"),
    F.round(F.min("consommation_clean"), 2).alias("minimum"),
    F.round(F.max("consommation_clean"), 2).alias("maximum"),
    F.round(F.expr("percentile_approx(consommation_clean, 0.25)"), 2).alias("Q1"),
    F.round(F.expr("percentile_approx(consommation_clean, 0.50)"), 2).alias("mediane"),
    F.round(F.expr("percentile_approx(consommation_clean, 0.75)"), 2).alias("Q3")
).orderBy("type_energie")

stats_par_energie.show(truncate=False)

Lignes avec valeur numerique valide : 7,719,893 / 7,758,868

STATISTIQUES DESCRIPTIVES PAR TYPE D'ENERGIE


+------------+----------+-------+----------+--------+--------+-----+-------+------+
|type_energie|nb_mesures|moyenne|ecart_type|minimum |maximum |Q1   |mediane|Q3    |
+------------+----------+-------+----------+--------+--------+-----+-------+------+
|eau         |2573156   |204.36 |2398.57   |-657.01 |49999.23|1.72 |7.52   |22.87 |
|electricite |2573364   |430.64 |2429.51   |-4003.35|49999.13|29.33|108.53 |306.34|
|gaz         |2573373   |560.94 |2465.81   |-5963.49|49999.49|43.63|160.57 |454.62|
+------------+----------+-------+----------+--------+--------+-----+-------+------+



## 4. Identification des problemes de qualite

Nous allons systematiquement identifier et quantifier chaque type de defaut present
dans le jeu de donnees :

1. **Valeurs textuelles** : lignes ou `consommation` n'est pas un nombre
2. **Valeurs negatives** : consommations < 0
3. **Valeurs aberrantes (outliers)** : consommations > 10 000
4. **Doublons** : lignes strictement identiques
5. **Separateurs decimaux mixtes** : utilisation de la virgule au lieu du point
6. **Formats de dates multiples** : ISO, FR, US, ISO avec T
7. **Periodes manquantes** : gaps temporels dans les series de mesures

In [6]:
# =============================================================================
# Identification et quantification des defauts de qualite
# =============================================================================

print("="*70)
print("AUDIT DE QUALITE DES DONNEES - IDENTIFICATION DES DEFAUTS")
print("="*70)
print(f"\nNombre total de lignes : {nb_total:,}\n")

# --- 1. Valeurs textuelles (non-numeriques) dans consommation ---
print("-"*70)
print("1. VALEURS TEXTUELLES (non-numeriques) dans 'consommation'")
print("-"*70)

df_texte = df_conso.filter(
    ~F.col("consommation").rlike(r"^-?[0-9]+([\.\,][0-9]+)?$")
)
nb_texte = df_texte.count()
pct_texte = (nb_texte / nb_total) * 100

print(f"  Nombre : {nb_texte:,} ({pct_texte:.2f}%)")
print("  Valeurs distinctes trouvees :")
df_texte.select("consommation").distinct().show(truncate=False)

# --- 2. Valeurs negatives ---
print("-"*70)
print("2. VALEURS NEGATIVES")
print("-"*70)

df_negatif = df_conso_num.filter(F.col("consommation_clean") < 0)
nb_negatif = df_negatif.count()
pct_negatif = (nb_negatif / nb_total) * 100

print(f"  Nombre : {nb_negatif:,} ({pct_negatif:.2f}%)")
print("  Exemples :")
df_negatif.select("batiment_id", "timestamp", "type_energie", "consommation_clean") \
    .orderBy("consommation_clean").show(5, truncate=False)

# --- 3. Valeurs aberrantes (outliers > 10 000) ---
print("-"*70)
print("3. VALEURS ABERRANTES (outliers > 10 000)")
print("-"*70)

df_outliers = df_conso_num.filter(F.col("consommation_clean") > 10000)
nb_outliers = df_outliers.count()
pct_outliers = (nb_outliers / nb_total) * 100

print(f"  Nombre : {nb_outliers:,} ({pct_outliers:.2f}%)")
print("  Distribution des outliers :")
df_outliers.select(
    F.round(F.min("consommation_clean"), 2).alias("min_outlier"),
    F.round(F.max("consommation_clean"), 2).alias("max_outlier"),
    F.round(F.mean("consommation_clean"), 2).alias("moyenne_outliers")
).show(truncate=False)

# --- 4. Doublons (lignes strictement identiques) ---
print("-"*70)
print("4. DOUBLONS (lignes strictement identiques)")
print("-"*70)

nb_distincts = df_conso.distinct().count()
nb_doublons = nb_total - nb_distincts
pct_doublons = (nb_doublons / nb_total) * 100

print(f"  Lignes distinctes : {nb_distincts:,}")
print(f"  Doublons          : {nb_doublons:,} ({pct_doublons:.2f}%)")

# Exemples de doublons
print("  Exemples de lignes dupliquees :")
df_conso.groupBy(df_conso.columns).count().filter(F.col("count") > 1) \
    .orderBy(F.col("count").desc()).show(5, truncate=False)

# --- 5. Separateurs decimaux mixtes (virgule) ---
print("-"*70)
print("5. SEPARATEURS DECIMAUX MIXTES (virgule au lieu de point)")
print("-"*70)

df_virgule = df_conso.filter(F.col("consommation").contains(","))
nb_virgule = df_virgule.count()
pct_virgule = (nb_virgule / nb_total) * 100

print(f"  Nombre : {nb_virgule:,} ({pct_virgule:.2f}%)")
print("  Exemples :")
df_virgule.select("consommation").show(5, truncate=False)

# --- 6. Formats de dates multiples ---
print("-"*70)
print("6. FORMATS DE DATES MULTIPLES")
print("-"*70)

# Detection des differents formats de date
df_dates = df_conso.withColumn(
    "format_date",
    F.when(
        F.col("timestamp").rlike(r"^\d{4}-\d{2}-\d{2}T\d{2}:\d{2}"),
        F.lit("ISO avec T (yyyy-MM-ddTHH:mm:ss)")
    ).when(
        F.col("timestamp").rlike(r"^\d{4}-\d{2}-\d{2} \d{2}:\d{2}"),
        F.lit("ISO standard (yyyy-MM-dd HH:mm:ss)")
    ).when(
        F.col("timestamp").rlike(r"^\d{2}/\d{2}/\d{4}"),
        F.lit("FR ou US (dd/mm/yyyy ou mm/dd/yyyy)")
    ).otherwise(
        F.lit("Autre / Inconnu")
    )
)

print("  Repartition des formats de dates :")
df_dates.groupBy("format_date").agg(
    F.count("*").alias("nb_lignes"),
    F.round(F.count("*") / nb_total * 100, 2).alias("pourcentage")
).orderBy(F.col("nb_lignes").desc()).show(truncate=False)

# Exemples pour chaque format
print("  Exemples par format :")
for fmt in df_dates.select("format_date").distinct().collect():
    print(f"\n  >> {fmt['format_date']} :")
    df_dates.filter(F.col("format_date") == fmt["format_date"]) \
        .select("timestamp").show(3, truncate=False)

# --- 7. Periodes de donnees manquantes ---
print("-"*70)
print("7. PERIODES DE DONNEES MANQUANTES")
print("-"*70)

# On analyse les valeurs nulles par colonne
print("  Valeurs nulles ou vides par colonne :")
for col_name in df_conso.columns:
    nb_nulls = df_conso.filter(
        F.col(col_name).isNull() | (F.trim(F.col(col_name).cast("string")) == "")
    ).count()
    pct_null = (nb_nulls / nb_total) * 100
    print(f"    {col_name:25s} : {nb_nulls:,} valeurs nulles/vides ({pct_null:.2f}%)")

# Detection de gaps temporels (periodes sans mesure)
# On prend un echantillon de batiments pour analyser la continuite
print("\n  Analyse des gaps temporels (echantillon de 5 batiments) :")

# On filtre les lignes avec des dates au format ISO standard pour cette analyse
df_iso = df_conso.filter(
    F.col("timestamp").rlike(r"^\d{4}-\d{2}-\d{2}")
).withColumn(
    "date_parsed",
    F.to_date(F.substring(F.col("timestamp"), 1, 10), "yyyy-MM-dd")
)

# Batiments avec le plus de mesures pour un echantillon representatif
top_batiments = df_iso.groupBy("batiment_id").count() \
    .orderBy(F.col("count").desc()).limit(5).select("batiment_id").collect()

for row in top_batiments:
    bid = row["batiment_id"]
    df_bat = df_iso.filter(F.col("batiment_id") == bid)
    date_min = df_bat.agg(F.min("date_parsed")).collect()[0][0]
    date_max = df_bat.agg(F.max("date_parsed")).collect()[0][0]
    nb_jours_distincts = df_bat.select("date_parsed").distinct().count()
    if date_min and date_max:
        nb_jours_attendus = (date_max - date_min).days + 1
        nb_jours_manquants = nb_jours_attendus - nb_jours_distincts
        print(f"    Batiment {bid} : {date_min} -> {date_max} | "
              f"{nb_jours_distincts} jours presents / {nb_jours_attendus} attendus | "
              f"{nb_jours_manquants} jours manquants")

print("\n" + "="*70)
print("RESUME DES DEFAUTS DETECTES")
print("="*70)
print(f"  {'Defaut':<45s} {'Nombre':>10s} {'%':>8s}")
print(f"  {'-'*45} {'-'*10} {'-'*8}")
print(f"  {'Valeurs textuelles':<45s} {nb_texte:>10,} {pct_texte:>7.2f}%")
print(f"  {'Valeurs negatives':<45s} {nb_negatif:>10,} {pct_negatif:>7.2f}%")
print(f"  {'Outliers (> 10 000)':<45s} {nb_outliers:>10,} {pct_outliers:>7.2f}%")
print(f"  {'Doublons':<45s} {nb_doublons:>10,} {pct_doublons:>7.2f}%")
print(f"  {'Separateurs decimaux mixtes (virgule)':<45s} {nb_virgule:>10,} {pct_virgule:>7.2f}%")

AUDIT DE QUALITE DES DONNEES - IDENTIFICATION DES DEFAUTS

Nombre total de lignes : 7,758,868

----------------------------------------------------------------------
1. VALEURS TEXTUELLES (non-numeriques) dans 'consommation'
----------------------------------------------------------------------


  Nombre : 38,975 (0.50%)
  Valeurs distinctes trouvees :


+------------+
|consommation|
+------------+
|null        |
|N/A         |
|erreur      |
|---         |
+------------+

----------------------------------------------------------------------
2. VALEURS NEGATIVES
----------------------------------------------------------------------


  Nombre : 38,910 (0.50%)
  Exemples :


+-----------+-------------------+------------+------------------+
|batiment_id|timestamp          |type_energie|consommation_clean|
+-----------+-------------------+------------+------------------+
|BAT0005    |2023-11-21T15:00:00|gaz         |-5963.49          |
|BAT0005    |03/11/2024 09:00   |gaz         |-5668.83          |
|BAT0005    |03/11/2024 09:00   |gaz         |-5668.83          |
|BAT0048    |06/11/2024 16:00   |gaz         |-5457.98          |
|BAT0005    |2023-11-14T10:00:00|gaz         |-5200.34          |
+-----------+-------------------+------------+------------------+
only showing top 5 rows
----------------------------------------------------------------------
3. VALEURS ABERRANTES (outliers > 10 000)
----------------------------------------------------------------------


...

    Batiment BAT0093 : 2023-01-01 -> 2024-12-31 | 731 jours presents / 731 attendus | 0 jours manquants


    Batiment BAT0047 : 2023-01-01 -> 2024-12-31 | 731 jours presents / 731 attendus | 0 jours manquants


    Batiment BAT0126 : 2023-01-01 -> 2024-12-31 | 731 jours presents / 731 attendus | 0 jours manquants


    Batiment BAT0045 : 2023-01-01 -> 2024-12-31 | 731 jours presents / 731 attendus | 0 jours manquants


    Batiment BAT0055 : 2023-01-01 -> 2024-12-31 | 731 jours presents / 731 attendus | 0 jours manquants

RESUME DES DEFAUTS DETECTES
  Defaut                                            Nombre        %
  --------------------------------------------- ---------- --------
  Valeurs textuelles                                38,975    0.50%
  Valeurs negatives                                 38,910    0.50%
  Outliers (> 10 000)                               38,560    0.50%
  Doublons                                         152,134    1.96%
  Separateurs decimaux mixtes (virgule)            925,392   11.93%


## 5. Batiments avec le plus de mesures

Analysons la distribution du nombre de mesures par batiment afin de verifier
l'homogeneite de la couverture des donnees.

In [7]:
# =============================================================================
# Nombre de mesures par batiment
# =============================================================================

df_mesures_par_bat = df_conso.groupBy("batiment_id").agg(
    F.count("*").alias("nb_mesures"),
    F.countDistinct("type_energie").alias("nb_types_energie"),
    F.min("timestamp").alias("premiere_mesure"),
    F.max("timestamp").alias("derniere_mesure")
).orderBy(F.col("nb_mesures").desc())

nb_batiments = df_mesures_par_bat.count()
print(f"Nombre total de batiments distincts : {nb_batiments}")

# Top 10 - Batiments avec le plus de mesures
print("\n" + "="*60)
print("TOP 10 - BATIMENTS AVEC LE PLUS DE MESURES")
print("="*60)
df_mesures_par_bat.show(10, truncate=False)

# Bottom 10 - Batiments avec le moins de mesures
print("\n" + "="*60)
print("BOTTOM 10 - BATIMENTS AVEC LE MOINS DE MESURES")
print("="*60)
df_mesures_par_bat.orderBy(F.col("nb_mesures").asc()).show(10, truncate=False)

# Statistiques sur la distribution des mesures par batiment
print("\n" + "="*60)
print("DISTRIBUTION DU NOMBRE DE MESURES PAR BATIMENT")
print("="*60)
df_mesures_par_bat.select("nb_mesures").summary(
    "count", "mean", "stddev", "min", "25%", "50%", "75%", "max"
).show(truncate=False)

Nombre total de batiments distincts : 146

TOP 10 - BATIMENTS AVEC LE PLUS DE MESURES


+-----------+----------+----------------+-------------------+----------------+
|batiment_id|nb_mesures|nb_types_energie|premiere_mesure    |derniere_mesure |
+-----------+----------+----------------+-------------------+----------------+
|BAT0086    |53275     |3               |01/01/2023 01:00   |31/12/2024 23:00|
|BAT0002    |53257     |3               |01/01/2023 01:00   |31/12/2024 22:00|
|BAT0145    |53255     |3               |01/01/2023 01:00   |31/12/2024 23:00|
|BAT0117    |53254     |3               |01/01/2023 00:00   |31/12/2024 23:00|
|BAT0047    |53254     |3               |01/01/2023 00:00:00|31/12/2024 22:00|
|BAT0051    |53246     |3               |01/01/2023 00:00   |31/12/2024 20:00|
|BAT0093    |53242     |3               |01/01/2023 00:00:00|31/12/2024 22:00|
|BAT0078    |53235     |3               |01/01/2023 00:00   |31/12/2024 23:00|
|BAT0146    |53235     |3               |01/01/2023 00:00:00|31/12/2024 23:00|
|BAT0097    |53233     |3               |01/01/2023 

+-----------+----------+----------------+-------------------+----------------+
|batiment_id|nb_mesures|nb_types_energie|premiere_mesure    |derniere_mesure |
+-----------+----------+----------------+-------------------+----------------+
|BAT0082    |53020     |3               |01/01/2023 00:00   |31/12/2024 21:00|
|BAT0140    |53024     |3               |01/01/2023 00:00   |31/12/2024 19:00|
|BAT0011    |53027     |3               |01/01/2023 00:00   |31/12/2024 23:00|
|BAT0048    |53034     |3               |01/01/2023 00:00:00|31/12/2024 23:00|
|BAT0074    |53041     |3               |01/01/2023 00:00   |31/12/2024 22:00|
|BAT0124    |53044     |3               |01/01/2023 00:00:00|31/12/2024 23:00|
|BAT0063    |53057     |3               |01/01/2023 00:00   |31/12/2024 23:00|
|BAT0136    |53059     |3               |01/01/2023 00:00:00|31/12/2024 22:00|
|BAT0115    |53060     |3               |01/01/2023 01:00   |31/12/2024 23:00|
|BAT0007    |53062     |3               |01/01/2023 

+-------+------------------+
|summary|nb_mesures        |
+-------+------------------+
|count  |146               |
|mean   |53142.931506849316|
|stddev |54.28566300019381 |
|min    |53020             |
|25%    |53108             |
|50%    |53142             |
|75%    |53175             |
|max    |53275             |
+-------+------------------+



## 6. Rapport d'audit de qualite des donnees

Nous construisons un rapport d'audit complet qui synthetise l'ensemble des constats
et propose des actions correctives pour chaque defaut identifie.

In [8]:
# =============================================================================
# Rapport d'audit complet de qualite des donnees
# =============================================================================

print("\n")
print("#" * 70)
print("#")
print("#   RAPPORT D'AUDIT DE QUALITE DES DONNEES")
print("#   ECF Energie - Consommations de batiments")
print("#")
print("#" * 70)

# --- Section 1 : Vue d'ensemble ---
print("\n" + "="*70)
print("  1. VUE D'ENSEMBLE DU JEU DE DONNEES")
print("="*70)
print(f"  Nombre total de lignes         : {nb_total:,}")
print(f"  Nombre de colonnes             : {len(df_conso.columns)}")
print(f"  Colonnes                       : {', '.join(df_conso.columns)}")
print(f"  Nombre de batiments distincts  : {nb_batiments}")

# Plage temporelle (a partir des dates ISO)
ts_min = df_conso.agg(F.min("timestamp")).collect()[0][0]
ts_max = df_conso.agg(F.max("timestamp")).collect()[0][0]
print(f"  Premiere mesure (min timestamp) : {ts_min}")
print(f"  Derniere mesure (max timestamp) : {ts_max}")

# Types d'energie
types_energie = [row[0] for row in df_conso.select("type_energie").distinct().collect()]
print(f"  Types d'energie                : {', '.join(sorted(types_energie))}")

# --- Section 2 : Defauts de qualite ---
print("\n" + "="*70)
print("  2. DEFAUTS DE QUALITE IDENTIFIES")
print("="*70)

# Construction du DataFrame de synthese des defauts
defauts = [
    ("Valeurs textuelles", nb_texte, pct_texte,
     "Champ consommation contient des textes : erreur, N/A, ---, null",
     "Supprimer ou remplacer par NULL puis interpoler si possible"),
    ("Valeurs negatives", nb_negatif, pct_negatif,
     "Consommations avec valeur inferieure a zero",
     "Prendre la valeur absolue ou supprimer selon le contexte"),
    ("Outliers (> 10 000)", nb_outliers, pct_outliers,
     "Pics de consommation anormalement eleves",
     "Appliquer un plafonnement (capping) ou supprimer"),
    ("Doublons", nb_doublons, pct_doublons,
     "Lignes strictement identiques presentes plusieurs fois",
     "Deduplication avec dropDuplicates()"),
    ("Separateurs decimaux mixtes", nb_virgule, pct_virgule,
     "Utilisation de la virgule au lieu du point comme separateur",
     "Remplacer les virgules par des points avant cast en double"),
    ("Formats de dates multiples", -1, -1.0,
     "Coexistence de formats ISO, FR (dd/mm/yyyy), US (mm/dd/yyyy), ISO+T",
     "Normaliser en un seul format ISO (yyyy-MM-dd HH:mm:ss)"),
    ("Periodes manquantes", -1, -1.0,
     "Gaps temporels dans les series de mesures de certains batiments",
     "Identifier les gaps et interpoler ou signaler les periodes manquantes")
]

# Affichage formate
print(f"\n  {'Defaut':<35s} {'Nombre':>10s} {'%':>8s}")
print(f"  {'-'*35} {'-'*10} {'-'*8}")
for defaut, nb, pct, desc, action in defauts:
    if nb >= 0:
        print(f"  {defaut:<35s} {nb:>10,} {pct:>7.2f}%")
    else:
        print(f"  {defaut:<35s} {'(multi)':>10s} {'(*)':>8s}")

# Details et actions recommandees
print("\n" + "="*70)
print("  3. DETAILS ET ACTIONS RECOMMANDEES")
print("="*70)

for i, (defaut, nb, pct, desc, action) in enumerate(defauts, 1):
    print(f"\n  {i}. {defaut}")
    print(f"     Description : {desc}")
    if nb >= 0:
        print(f"     Impact      : {nb:,} lignes ({pct:.2f}% du total)")
    else:
        print(f"     Impact      : Multiple formats / periodes detectes")
    print(f"     Action      : {action}")

# --- Section 3 : Score de qualite global ---
print("\n" + "="*70)
print("  4. SCORE DE QUALITE GLOBAL")
print("="*70)

# Calcul du nombre total de lignes affectees par au moins un defaut
# (estimation conservative : on somme les defauts quantifies)
nb_defauts_quantifies = nb_texte + nb_negatif + nb_outliers + nb_doublons + nb_virgule
# Les separateurs decimaux ne sont pas des "erreurs" a supprimer, juste a transformer
# On exclut donc les virgules du calcul du score de qualite
nb_lignes_problematiques = nb_texte + nb_negatif + nb_outliers + nb_doublons
pct_problematique = (nb_lignes_problematiques / nb_total) * 100
score_qualite = 100 - pct_problematique

print(f"\n  Lignes problematiques (hors format) : {nb_lignes_problematiques:,} ({pct_problematique:.2f}%)")
print(f"  Lignes necessitant une transformation : {nb_virgule:,} (separateurs decimaux)")
print(f"")
print(f"  +{'='*50}+")
print(f"  |  SCORE DE QUALITE GLOBAL : {score_qualite:.1f} / 100         |")
print(f"  +{'='*50}+")

if score_qualite >= 90:
    appreciation = "BON - Les donnees sont exploitables apres nettoyage mineur"
elif score_qualite >= 75:
    appreciation = "MOYEN - Un nettoyage significatif est necessaire"
else:
    appreciation = "FAIBLE - Un nettoyage en profondeur est indispensable"

print(f"  Appreciation : {appreciation}")

# --- Section 4 : Etapes suivantes ---
print("\n" + "="*70)
print("  5. ETAPES SUIVANTES DU PIPELINE")
print("="*70)
print("")
print("  1. Nettoyage des donnees (notebook 02) :")
print("     - Normalisation des formats de dates")
print("     - Conversion des separateurs decimaux")
print("     - Suppression des doublons")
print("     - Traitement des valeurs textuelles")
print("     - Gestion des valeurs negatives et outliers")
print("")
print("  2. Enrichissement et jointure avec batiments.csv")
print("  3. Analyse approfondie et visualisations")
print("  4. Modelisation et predictions")
print("\n" + "#"*70)



######################################################################
#
#   RAPPORT D'AUDIT DE QUALITE DES DONNEES
#   ECF Energie - Consommations de batiments
#
######################################################################

  1. VUE D'ENSEMBLE DU JEU DE DONNEES
  Nombre total de lignes         : 7,758,868
  Nombre de colonnes             : 5
  Colonnes                       : batiment_id, timestamp, type_energie, consommation, unite
  Nombre de batiments distincts  : 146


  Premiere mesure (min timestamp) : 01/01/2023 00:00
  Derniere mesure (max timestamp) : 31/12/2024 23:00


  Types d'energie                : eau, electricite, gaz

  2. DEFAUTS DE QUALITE IDENTIFIES

  Defaut                                  Nombre        %
  ----------------------------------- ---------- --------
  Valeurs textuelles                      38,975    0.50%
  Valeurs negatives                       38,910    0.50%
  Outliers (> 10 000)                     38,560    0.50%
  Doublons                               152,134    1.96%
  Separateurs decimaux mixtes            925,392   11.93%
  Formats de dates multiples             (multi)      (*)
  Periodes manquantes                    (multi)      (*)

  3. DETAILS ET ACTIONS RECOMMANDEES

  1. Valeurs textuelles
     Description : Champ consommation contient des textes : erreur, N/A, ---, null
     Impact      : 38,975 lignes (0.50% du total)
     Action      : Supprimer ou remplacer par NULL puis interpoler si possible

  2. Valeurs negatives
     Description : Consommations avec valeur inferieure a zero
     Impact      : 38,

## Conclusion

### Synthese de l'exploration initiale

L'exploration du jeu de donnees `consommations_raw.csv` a permis de mettre en evidence
les constats suivants :

**Structure des donnees :**
- Le fichier contient environ 7,7 millions de mesures de consommation energetique
- Les mesures concernent 146 batiments et couvrent plusieurs types d'energie
- La colonne `consommation` est inferee en `string` par PySpark en raison de la
  presence de valeurs textuelles et de separateurs decimaux mixtes

**Problemes de qualite identifies :**
- **Valeurs textuelles** (~1,5%) : des chaines comme "erreur", "N/A", "---", "null"
  polluent le champ consommation
- **Separateurs decimaux** (~12%) : une part significative utilise la virgule au lieu du point
- **Doublons** (~2%) : des lignes strictement identiques sont presentes
- **Valeurs negatives** (~0,5%) : des consommations negatives (physiquement impossibles)
- **Outliers** (~1%) : des pics de consommation anormalement eleves (> 10 000)
- **Formats de dates multiples** : coexistence de 4 formats differents
- **Periodes manquantes** : gaps temporels identifies dans certaines series

**Prochaine etape :**
Le notebook suivant mettra en oeuvre le pipeline de nettoyage pour corriger ces defauts
et produire un jeu de donnees propre et exploitable pour les analyses ulterieures.

In [9]:
# =============================================================================
# Fermeture de la session Spark
# =============================================================================

spark.stop()
print("Session Spark fermee avec succes.")

Session Spark fermee avec succes.
